In [1]:
from z3 import *

In [2]:
# 阶段
Stage, stage_consts = EnumSort('Stage', ['PREPARE', 'CONFIRM', 'EXTERNALIZE'])
# 候选值
Cand = IntSort()
# ballot
Ballot_Nil, ballot_nil = EnumSort('Ballot_Nil', ['ballot_nil'])
ballot_nil = ballot_nil[0]

Ballot = Datatype('Ballot')
Ballot.declare('cons', ('n', IntSort()), ('x', Cand))
Ballot.declare('nil')
Ballot = Ballot.create()
ballot_nil = Ballot.nil

# 节点状态 记录phi b p' p c h z 暂时不记录M
Nstate = Datatype('Nstate')
Nstate.declare('cons', ('phi', Stage), ('b', Ballot), ('p1', Ballot), ('p', Ballot), ('c', Ballot), ('h', Ballot), ('z', Cand))
Nstate = Nstate.create()


In [4]:
# 时间步长
T = 1000
times = [f'node{i}' for i in range(T)]
Time, time_consts = EnumSort('Time', times)

In [ ]:
node_state = Function('node_state', Node, Time, Nstate) # 每个节点的状态

In [ ]:
Init = [
    ForAll([node1], node_state(node1, times[0]).phi == stage_consts[0]),
    ForAll([node1], And(node_state(node1, times[0]).b.n == stage_consts[0], node_state(node1, times[0]).b.n == node_state(node1, times[0]).z)),
    ForAll([node1], node_state(node1, times[0]).p1 == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).p == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).c == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).h == ballot_nil),
]

CS = [
    ForAll([node1, time], Or(node_state(node1, time).p == ballot_nil, And(accept_prepared(node1, node_state(node1, time).p, time), ForAll([ballot], Implies(accept_prepared(node1, ballot, time), ballot.n <= node_state(node1, time).p))))),
    ForAll([node1, time], Or(node_state(node1, time).p1 == ballot_nil, And(accept_prepared(node1, node_state(node1, time).p1, time), node_state(node1, time).p.x != node_state(node1, time).p1.x, ForAll([ballot], Implies(accept_prepared(node1, ballot, time), ballot.n < node_state(node1, time).p))))),
    ForAll([node1, time], Implies(And(node_state(node1, time).phi == stage_consts[0], node_state(node1, time).c == ballot.nil), Or(node_state(node1, time).h == ballot_nil, And(confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), ballot.n <= node_state(node1, time).h)))))),
    ForAll([node1, time], Implies(And(node_state(node1, time).phi == stage_consts[0], node_state(node1, time).c != ballot.nil), And(confirm_prepared(node1, node_state(node1, time).c), confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).phi == stage_consts[1], And(accept_prepared(node1, node_state(node1, time).c, time), accept_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(accept_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).phi == stage_consts[2], And(confirm_prepared(node1, node_state(node1, time).c, time), confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).h != ballot, node_state(node1, time).h.x == node_state(node1, time).z)),
]

In [ ]:
# 对节点状态的限制 c<~h<~b
C1 = [
    ForAll([node1, time], And(node_state(node1, time).c.x == node_state(node1, time).h.x, node_state(node1, time).h.x == node_state(node1, time).b.x)),
    ForAll([node1, time], And(node_state(node1, time).c.n <= node_state(node1, time).h.x, node_state(node1, time).h.x <= node_state(node1, time).b.x)),
]

In [ ]:
# 状态转移规则
C2_pre = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(Exists([ballot], And(Not(accept_prepared(node1, ballot, time_consts[t-1])), accept_prepared(node1, ballot, time_consts[t]))), node_state(node1, time_consts[t+1]).c == ballot_nil))) for t in range(1, T)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(And(node_state(node1, time_consts[t-1]).c == ballot_nil, node_state(node1, time_consts[t-1]).b.n <= node_state(node1, time_consts[t]).h.n, Not(And(node_state(node1, time_consts[t-1]).p.n >= node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t-1]).p.x != node_state(node1, time_consts[t]).h.x)), Not(And(node_state(node1, time_consts[t-1]).p1.n >= node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t-1]).p1.x != node_state(node1, time_consts[t]).h.x))), And(node_state(node1, time_consts[t]).c.n >= node_state(node1, time_consts[t-1]).b.n, ForAll([ballot], Implies(And(ballot.n >= node_state(node1, time_consts[t-1]).b.n, ballot.n <= node_state(node1, time_consts[t]).h.n, ballot.x == node_state(node1, time_consts[t]).h.x), ballot.n <= node_state(node1, time_consts[t]).c.n)))))) for t in range(1, T+1)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(Exists([ballot], accept_commit(node1, ballot, time_consts[t-1])), And(node_state(node1, time_consts[t]).phi == stage_consts[1], Implies(Not(And(node_state(node1, time_consts[t]).h.n <= node_state(node1, time_consts[t-1]).b.n, node_state(node1, time_consts[t]).h.x == node_state(node1, time_consts[t-1]).b.x)), And(node_state(node1, time_consts[t]).b == node_state(node1, time_consts[t]).x, node_state_6(node1, t))))))) for t in range(1, T+1)],
]

C2_con = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[1], Implies(Or(Exists([ballot], And(Not(accept_commit(node1, ballot, time_consts[t-1])), accept_commit(node1, ballot, time_consts[t]))), node_state_6(node1, time_consts[t])), And(Implies(node_h1_6(node1, time_consts[t]).n > node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t+1]).h == node_h1_6(node1, time_consts[t])), node_state(node1, time_consts[t+1]).c == node_c_6(node1, time_consts[t]))))) for t in range(1, T)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[1], Implies(Exists([ballot], confirm_commit(node1, ballot, time_consts[t-1])), node_state(node1, time_consts[t]).phi == stage_consts[2]))) for t in range(1, T+1)],
]

# terminate
C2_ext = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[2], node_state(node1, time_consts[t-1]) == node_state(node1, time_consts[t]))) for t in range(1, T+1)],
]

C2_raise_b = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).b.n < node_state(node1, time_consts[t]).h.n, node_state_6(node1, time_consts[t]))) for t in range(1, T+1)],
]

C2_deblock = [
    Forall([node1, n, mset, time], Implies(And(is_blocking_signal(node1, mset), ForAll([msg], Implies(in_mset(msg, mset), msg.b.n > n))), deblock_S_over_n(node1, n, mset, time))), 
    [ForAll([node1], Implies(Exists([mset], deblock_S_over_n(node1, node_state(node1, time_consts[t]).b.n, mset, time_consts[t-1])), Exists([n], And(node_state(node1, time_consts[t]).b == ballot_con(n, node_state(node1, time_consts[t-1]).z), Not(Exists([mset], deblock_S_over_n(node1, n, mset, time_consts[t-1]))))))) for t in range(1, T+1)],
]

In [ ]:
# 规则约束
# 初始值
Init = [
    ForAll([node1], node_state(node1, times[0]).phi == stage_consts[0]),
    ForAll([node1], And(node_state(node1, times[0]).b.n == stage_consts[0], node_state(node1, times[0]).b.n == node_state(node1, times[0]).z)),
    ForAll([node1], node_state(node1, times[0]).p1 == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).p == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).c == ballot_nil),
    ForAll([node1], node_state(node1, times[0]).h == ballot_nil),
]
# 节点状态的约束
CS = [
    ForAll([node1, time], Or(node_state(node1, time).p == ballot_nil, And(accept_prepared(node1, node_state(node1, time).p, time), ForAll([ballot], Implies(accept_prepared(node1, ballot, time), ballot.n <= node_state(node1, time).p))))),
    ForAll([node1, time], Or(node_state(node1, time).p1 == ballot_nil, And(accept_prepared(node1, node_state(node1, time).p1, time), node_state(node1, time).p.x != node_state(node1, time).p1.x, ForAll([ballot], Implies(accept_prepared(node1, ballot, time), ballot.n < node_state(node1, time).p))))),
    ForAll([node1, time], Implies(And(node_state(node1, time).phi == stage_consts[0], node_state(node1, time).c == ballot.nil), Or(node_state(node1, time).h == ballot_nil, And(confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), ballot.n <= node_state(node1, time).h)))))),
    ForAll([node1, time], Implies(And(node_state(node1, time).phi == stage_consts[0], node_state(node1, time).c != ballot.nil), And(confirm_prepared(node1, node_state(node1, time).c), confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).phi == stage_consts[1], And(accept_prepared(node1, node_state(node1, time).c, time), accept_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(accept_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).phi == stage_consts[2], And(confirm_prepared(node1, node_state(node1, time).c, time), confirm_prepared(node1, node_state(node1, time).h, time), ForAll([ballot], Implies(confirm_prepared(node1, ballot, time), And(ballot.n <= node_state(node1, time).h, ballot.n <= node_state(node1, time).c)))))),
    ForAll([node1, time], Implies(node_state(node1, time).h != ballot, node_state(node1, time).h.x == node_state(node1, time).z)),
]
# 对节点状态的限制 c<~h<~b
C1 = [
    ForAll([node1, time], And(node_state(node1, time).c.x == node_state(node1, time).h.x, node_state(node1, time).h.x == node_state(node1, time).b.x)),
    ForAll([node1, time], And(node_state(node1, time).c.n <= node_state(node1, time).h.x, node_state(node1, time).h.x <= node_state(node1, time).b.x)),
]
# 状态转移规则
C2_pre = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(Exists([ballot], And(Not(accept_prepared(node1, ballot, time_consts[t-1])), accept_prepared(node1, ballot, time_consts[t]))), node_state(node1, time_consts[t+1]).c == ballot_nil))) for t in range(1, T)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(And(node_state(node1, time_consts[t-1]).c == ballot_nil, node_state(node1, time_consts[t-1]).b.n <= node_state(node1, time_consts[t]).h.n, Not(And(node_state(node1, time_consts[t-1]).p.n >= node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t-1]).p.x != node_state(node1, time_consts[t]).h.x)), Not(And(node_state(node1, time_consts[t-1]).p1.n >= node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t-1]).p1.x != node_state(node1, time_consts[t]).h.x))), And(node_state(node1, time_consts[t]).c.n >= node_state(node1, time_consts[t-1]).b.n, ForAll([ballot], Implies(And(ballot.n >= node_state(node1, time_consts[t-1]).b.n, ballot.n <= node_state(node1, time_consts[t]).h.n, ballot.x == node_state(node1, time_consts[t]).h.x), ballot.n <= node_state(node1, time_consts[t]).c.n)))))) for t in range(1, T+1)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[0], Implies(Exists([ballot], accept_commit(node1, ballot, time_consts[t-1])), And(node_state(node1, time_consts[t]).phi == stage_consts[1]), Implies(Not(And(node_state(node1, time_consts[t]).h.n <= node_state(node1, time_consts[t-1]).b.n, node_state(node1, time_consts[t]).h.x == node_state(node1, time_consts[t-1]).b.x)), And(node_state(node1, time_consts[t]).b == node_state(node1, time_consts[t]).x, node_state_6(node1, t)))))) for t in range(1, T+1)],
]
C2_con = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[1], Implies(Or(Exists([ballot], And(Not(accept_commit(node1, ballot, time_consts[t-1])), accept_commit(node1, ballot, time_consts[t]))), node_state_6(node1, time_consts[t])), And(Implies(node_h1_6(node1, time_consts[t]).n > node_state(node1, time_consts[t]).h.n, node_state(node1, time_consts[t+1]).h == node_h1_6(node1, time_consts[t])), node_state(node1, time_consts[t+1]).c == node_c_6(node1, time_consts[t]))))) for t in range(1, T)],
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[1], Implies(Exists([ballot], confirm_commit(node1, ballot, time_consts[t-1])), node_state(node1, time_consts[t]).phi == stage_consts[2]))) for t in range(1, T+1)],
]
C2_ext = [ # terminate
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).phi == stage_consts[2], node_state(node1, time_consts[t-1]) == node_state(node1, time_consts[t]))) for t in range(1, T+1)],
]
C2_raise_b = [
    [ForAll([node1], Implies(node_state(node1, time_consts[t-1]).b.n < node_state(node1, time_consts[t]).h.n, node_state_6(node1, time_consts[t]))) for t in range(1, T+1)],
]
C2_deblock = [
    ForAll([node1, n, mset, time], Implies(And(is_blocking_signal(node1, mset), ForAll([msg], Implies(in_mset(msg, mset), msg.b.n > n))), deblock_S_over_n(node1, n, mset, time))), 
    [ForAll([node1], Implies(Exists([mset], deblock_S_over_n(node1, node_state(node1, time_consts[t]).b.n, mset, time_consts[t-1])), Exists([n], And(node_state(node1, time_consts[t]).b == ballot_con(n, node_state(node1, time_consts[t-1]).z), Not(Exists([mset], deblock_S_over_n(node1, n, mset, time_consts[t-1]))))))) for t in range(1, T+1)],
]